In [6]:
import pandas as pd
import numpy as np



import os
import sqlite3
import kagglehub
import warnings

warnings.filterwarnings('ignore')   

In [16]:
# Download dataset from kaggle
path = kagglehub.dataset_download("rtatman/188-million-us-wildfires")
path = os.path.join(path,os.listdir(path)[0])
print(path)

C:\Users\hyder\.cache\kagglehub\datasets\rtatman\188-million-us-wildfires\versions\2\FPA_FOD_20170508.sqlite


In [17]:
conn = sqlite3.connect(path)
type(conn)

sqlite3.Connection

In [26]:
# Step 2: Query sqlite_master to get all table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Step 3: Fetch all table names
tables = cursor.fetchall()

# Step 4: Print the table names
print("Tables in the SQLite database:")
for table in tables:
    print(table[0])

Tables in the SQLite database:
spatial_ref_sys
spatialite_history
sqlite_sequence
geometry_columns
spatial_ref_sys_aux
views_geometry_columns
virts_geometry_columns
geometry_columns_statistics
views_geometry_columns_statistics
virts_geometry_columns_statistics
geometry_columns_field_infos
views_geometry_columns_field_infos
virts_geometry_columns_field_infos
geometry_columns_time
geometry_columns_auth
views_geometry_columns_auth
virts_geometry_columns_auth
sql_statements_log
SpatialIndex
ElementaryGeometries
KNN
Fires
idx_Fires_Shape
idx_Fires_Shape_node
idx_Fires_Shape_rowid
idx_Fires_Shape_parent
NWCG_UnitIDActive_20170109


In [27]:
# Querying data from sql database
df = pd.read_sql_query(
    "SELECT * FROM Fires",
    conn
    )